In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [23]:
train_dir = "D:/robo-sumr-2025/project/data/asl_alphabet_train"
val_dir   = "D:/robo-sumr-2025/project/data/asl_alphabet_test"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

test_datagen = ImageDataGenerator(rescale=1./255)

val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical"
)


Found 67200 images belonging to 28 classes.
Found 28 images belonging to 28 classes.


In [24]:
from keras.src.applications.mobilenet_v2 import MobileNetV2

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(128,128,3))
base_model.trainable = False


In [25]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
preds = Dense(train_generator.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=preds)


In [26]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])


In [27]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


Epoch 1/10
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 629s 297ms/step - accuracy: 0.3024 - loss: 2.5704 - val_accuracy: 0.9286 - val_loss: 0.5573
Epoch 2/10
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 608s 290ms/step - accuracy: 0.5644 - loss: 1.4418 - val_accuracy: 0.9643 - val_loss: 0.3171
Epoch 3/10
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 655s 312ms/step - accuracy: 0.6539 - loss: 1.1453 - val_accuracy: 0.9643 - val_loss: 0.2331
Epoch 4/10
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 650s 309ms/step - accuracy: 0.7005 - loss: 0.9931 - val_accuracy: 0.9643 - val_loss: 0.1951
Epoch 5/10
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 689s 328ms/step - accuracy: 0.7273 - loss: 0.9025 - val_accuracy: 0.9643 - val_loss: 0.1768
Epoch 6/10
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 676s 322ms/step - accuracy: 0.7438 - loss: 0.8464 - val_accuracy: 0.9643 - val_loss: 0.1596
Epoch 7/10
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 638s 304ms/step - accuracy: 0.7518 - loss: 0.8195 - val_accuracy: 0.9643 - val_loss: 0.1516
Epoch 8/10
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 671s 320ms/step - ac

In [28]:
model.save("my_model.h5")

In [29]:
#########################
import numpy as np
from tensorflow.keras.preprocessing import image

class_labels = list(train_generator.class_indices.keys())

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(128,128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)
    predicted_class = np.argmax(preds, axis=1)[0]
    confidence = np.max(preds)

    return class_labels[predicted_class], confidence


In [46]:
test_img = "D:/robo-sumr-2025/project/data/asl_alphabet_test/A/A_test.jpg"

test_img2 = "C:/Users/zyads/Desktop/NN.jpg"

label, conf = predict_image(test_img2)
print(f"Predicted: {label} ({conf*100:.2f}% confidence)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted: S (66.80% confidence)
